In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [3]:
#data with feature engineering
fields = ['Date', 'Open','High','Low','Close','Volume','RSI','MACD','Ichimoku_Span_A','Ichimoku_Span_B','ATR',
         'BB_High_Indicator','BB_Low_Indicator']
AAPL_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AAPL.csv", usecols=fields)

In [4]:
AAPL_df['Date'] = pd.to_datetime(AAPL_df.Date, format ="%Y-%m-%d")

In [5]:
AAPL_2010 = pd.read_csv("/Users/fahad/project_repo/data/final/AAPL/AAPL_10yr.csv")


In [6]:
AAPL_2010['Date'] = pd.to_datetime(AAPL_2010['Date'], format = "%Y-%m-%d")
AAPL_2010['Date_']= AAPL_2010['Date']
AAPL_2010 = AAPL_2010.set_index("Date")

In [7]:
AAPL_2010.dtypes

Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
Date_     datetime64[ns]
dtype: object

In [12]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
AAPL_2010_labels = labelling(AAPL_2010)

In [13]:
AAPL_2010_flattened= get_ndays_monthly_flattened(AAPL_2010,AAPL_df,30)


In [14]:
AAPL_2010_labels_df = pd.Series(AAPL_2010_labels).to_frame("Label")

In [15]:
#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
AAPL_2010_flat_df = pd.DataFrame.from_dict(AAPL_2010_flattened).T


In [16]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
AAPL_2010_flat_df = AAPL_2010_flat_df.merge(AAPL_2010_labels_df, left_index=True, right_index=True)


In [17]:
export = AAPL_2010_flat_df.to_csv("/Users/fahad/project_repo/data/final/AAPL/AAPL_2010_pre_signals.csv")

In [9]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [10]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    